In [1]:
import pandas as pd
import bellykit

import ffn
import talib

In [2]:
engine = bellykit.bellykit_sql.login_guest('stock')
def get_price(*args):
    for symbol in args:
        price = bellykit.data_api.get_price(str(symbol), select=['DATE', 'OPEN', 'HIGH', 'LOW','CLOSE', 'VOLUME'], start='2012-01-01')
        price.columns = price.columns.str.lower()
        price.set_index('date', drop=True, inplace=True)
        col = [(symbol, i) for i in price.columns.to_list()]
        price.columns = pd.MultiIndex.from_tuples(col)
        try:
            data = pd.merge(left=data, right=price, on='date')
        except NameError:
            data = price
        
    return data

# 生成指標
1. 自動併入data中

In [3]:
def MA(symbol, period, price='close'):
    global data

    name = f'MA{period}'
    ma_series = data[symbol, price].rolling(period).mean().fillna(0)
    ma_df = pd.DataFrame(ma_series)
    ma_df.columns = pd.MultiIndex.from_tuples([(symbol, name)])
    data[(symbol, name)] = ma_df
    
def crossover(symbol, ind1, ind2, price='close'):
    global data
    name = f'{ind1}_x_{ind2}'
    c1 = data[symbol, ind1] > data[symbol, ind2]
    c2 = (~c1).shift(1, fill_value=False)
    crossover_series = (c1&c2).astype(int)
    data[symbol, name] = crossover_series

def RSI(symbol, period, price='close'):
    global data
    name = f'RSI({period})'
    rsi_series = talib.RSI(data[2330,'close'], timeperiod=period).fillna(0)
    data[symbol, name] = rsi_series
    pass

In [4]:
data = get_price(2330)

In [5]:
data

2330                               
             open   high    low  close    volume
date                                            
2012-01-02   75.0   75.4   74.7   75.0  14945219
2012-01-03   75.5   75.9   75.2   75.7  21223640
2012-01-04   76.1   76.1   75.5   75.8  34126472
2012-01-05   76.0   76.0   75.3   76.0  28076670
2012-01-06   75.9   75.9   74.9   75.6  30123118
...           ...    ...    ...    ...       ...
2020-09-23  436.0  438.5  432.0  433.5  45710963
2020-09-24  425.5  429.0  423.0  423.0  79806381
2020-09-25  427.0  428.0  421.0  424.0  38775269
2020-09-28  427.0  431.5  424.5  431.5  34156973
2020-09-29  432.5  435.0  428.0  431.0  32230502

[2148 rows x 5 columns]

In [6]:
MA(2330, 5)
MA(2330, 10)

RSI(2330, 5)
RSI(2330, 15)
crossover(2330, 'RSI(5)', 'RSI(15)')
crossover(2330, 'MA5', 'MA10')

In [7]:
order_list = []


def new_order(target, **kwargs):
    order_list.append(Order(target, **kwargs))


def handle_order():
    account.passed()
    
    global order_list
    while True:
        try:
            order = order_list.pop(0)
        except IndexError:
            break
        account.deal_order(order)
        
    
    #check exit
    to_del_ind = []
    for ind, holding in enumerate(holding_list):
        if holding.last == 0:
            print('entry/exit date:', holding.entry_date.date(), date.date(), holding.target, 'entry/exit price:', holding.price, '/',row[holding.target, 'open'], holding.describe)
            to_del_ind.append(ind)
    
    for index in sorted(to_del_ind, reverse=True):
        del holding_list[index]

In [8]:

class Stock():
    class Trade():
        def __init__(self, entry_time, entry_price, share):
            self.entry_time = entry_time
            self.entry_price = entry_price
            self.share = share
    
    def __init__(self, target):
        self.target = target
        self.trades = []
        
    def buy(self, dealed_order):
        self.trades.append(dealed_order)

        
class Order():
    def __init__(self, target, **kwargs):
        self.target = target
        
        self.last = kwargs.get('last', -1)
        
        self.describe = kwargs.get('describe', None)
        self.stoploss = kwargs.get('stoploss', None)
        self.stopprofit = kwargs.get('stopprofit', None)
        
        self.share = None
        self.entry_price = None
        self.entry_date = None
        self.exit_price = None
        self.exit_date = None
        
        self._adj_entry_price = None
        self._adj_cover_price = None
        
        
        
class Account():
    def __init__(self, equity):
        self.equity = equity
        self.hold = None
        
        self.order = None
    
    def empty(self):
        if self.hold == None:
            return True
        else:
            return False
        
        
    def buy(self, target, **kwargs):
        if self.order is None:
            new_order = Order(target, **kwargs)
            self.order = new_order
        else:
            print(f'已經有待處理的委託（{target}）。')
        
    def head(self, date, row):
        print(date.date())

        if self.hold is not None:
            self.hold.last -= 1
        
        if self.order is not None:
            # 已經有持倉，不受理或是延長
            if self.hold is not None:
                if self.hold.target != self.order.target:
                    print('餘額不足，委託不受理')
                else:
                    print('延長標的持有時間')
                    self.hold.last = self.order.last
                    self.order = None
            # 新標的
            else:
                self.hold = self.order
                
                entry_price = row[self.order.target, 'open']
                entry_date = date
                share = int(self.equity / entry_price)
                cost = share * entry_price
                self.equity -= cost
                
                self.hold.entry_price = entry_price
                self.hold.share = share
                self.hold.entry_date = entry_date

                print(f'date: {date.date()}, 買進股票{self.order.target}, price: entry_price')
                self.order = None

        if self.hold is None:
            return
        
        
        # 到期 賣出
        if self.hold.last == 0:
            hold = self.hold
            exit_price = row[self.hold.target, 'open']
            exit_date = date
            
            equity = exit_price * self.hold.share
            equity_change = (exit_price - self.hold.entry_price) * self.hold.share
            self.equity += equity
            print(hold.target, hold.entry_date, exit_date, ', entry/exit price:', hold.entry_price, exit_price)
            print('equity change:', equity_change)
            
            self.hold = None
            
    def exitall(self, date, row):
        if self.hold is None:
            return
        hold = self.hold
        exit_price = row[self.hold.target, 'open']
        exit_date = date

        equity = exit_price * self.hold.share
        equity_change = (exit_price - self.hold.entry_price) * self.hold.share
        self.equity += equity
        print(hold.target, hold.entry_date, exit_date, ', entry/exit price:', hold.entry_price, exit_price)
        print('equity change:', equity_change)

        self.hold = None


In [9]:
# equity = 100

order = None
hold = None

account = Account(equity=1000000)

pre_2330_ma5 = 0
pre_2330_ma10 = 0


buy_next = False


for date, row in data.iterrows():
    account.head(date, row)
    
    if row[2330, 'RSI(5)_x_RSI(15)']:
        print('2330 RSI crossover')
        account.buy(2330, last=10, stoploss=0.08)
            
    if row[2330, 'MA5_x_MA10']:
        print('2330 ma crossover')
        account.buy(2330, last=10, stoploss=0.08)

account.exitall(date, data.iloc[-1])

2012-01-02
2012-01-03
2012-01-04
2012-01-05
2012-01-06
2330 ma crossover
2012-01-09
date: 2012-01-09, 買進股票2330, price: entry_price
2330 RSI crossover
2012-01-10
延長標的持有時間
2012-01-11
2012-01-12
2012-01-13
2012-01-16
2012-01-17
2012-01-18
2012-01-30
2012-01-31
2012-02-01
2012-02-02
2330 2012-01-09 00:00:00 2012-02-02 00:00:00 , entry/exit price: 75.2 77.5
equity change: 30583.099999999962
2012-02-03
2012-02-04
2012-02-06
2012-02-07
2012-02-08
2330 RSI crossover
2012-02-09
date: 2012-02-09, 買進股票2330, price: entry_price
2012-02-10
2330 ma crossover
2012-02-13
延長標的持有時間
2012-02-14
2012-02-15
2330 RSI crossover
2012-02-16
延長標的持有時間
2012-02-17
2012-02-20
2012-02-21
2012-02-22
2012-02-23
2012-02-24
2012-02-29
2330 RSI crossover
2012-03-01
延長標的持有時間
2012-03-02
2012-03-03
2012-03-05
2012-03-06
2012-03-07
2012-03-08
2012-03-09
2330 RSI crossover
2012-03-12
延長標的持有時間
2012-03-13
2330 ma crossover
2012-03-14
延長標的持有時間
2012-03-15
2012-03-16
2012-03-19
2330 RSI crossover
2012-03-20
延長標的持有時間
2012-03-21
2012-

2013-12-25
2330 ma crossover
2013-12-26
延長標的持有時間
2013-12-27
2013-12-30
2013-12-31
2014-01-02
2014-01-03
2014-01-06
2014-01-07
2014-01-08
2330 RSI crossover
2014-01-09
延長標的持有時間
2014-01-10
2014-01-13
2330 RSI crossover
2014-01-14
延長標的持有時間
2014-01-15
2014-01-16
2330 ma crossover
2014-01-17
延長標的持有時間
2014-01-20
2014-01-21
2014-01-22
2014-01-23
2014-01-24
2014-01-27
2014-02-05
2014-02-06
2014-02-07
2014-02-10
2330 2013-11-20 00:00:00 2014-02-10 00:00:00 , entry/exit price: 104.0 104.0
equity change: 0.0
2014-02-11
2330 RSI crossover
2014-02-12
date: 2014-02-12, 買進股票2330, price: entry_price
2014-02-13
2014-02-14
2330 ma crossover
2014-02-17
延長標的持有時間
2014-02-18
2014-02-19
2014-02-20
2014-02-21
2330 RSI crossover
2014-02-24
延長標的持有時間
2014-02-25
2330 RSI crossover
2014-02-26
延長標的持有時間
2014-02-27
2014-03-03
2014-03-04
2330 ma crossover
2014-03-05
延長標的持有時間
2330 RSI crossover
2014-03-06
延長標的持有時間
2014-03-07
2014-03-10
2014-03-11
2014-03-12
2014-03-13
2330 RSI crossover
2014-03-14
延長標的持有時間
2014-03-17
2

2016-07-19
2016-07-20
2016-07-21
2016-07-22
2016-07-25
2330 2016-06-16 00:00:00 2016-07-25 00:00:00 , entry/exit price: 161.5 172.0
equity change: 117652.5
2016-07-26
2016-07-27
2016-07-28
2016-07-29
2016-08-01
2330 RSI crossover
2016-08-02
date: 2016-08-02, 買進股票2330, price: entry_price
2016-08-03
2016-08-04
2016-08-05
2330 RSI crossover
2016-08-08
延長標的持有時間
2016-08-09
2016-08-10
2016-08-11
2016-08-12
2016-08-15
2330 RSI crossover
2016-08-16
延長標的持有時間
2016-08-17
2016-08-18
2016-08-19
2016-08-22
2016-08-23
2016-08-24
2016-08-25
2330 RSI crossover
2016-08-26
延長標的持有時間
2016-08-29
2330 ma crossover
2016-08-30
延長標的持有時間
2016-08-31
2016-09-01
2016-09-02
2016-09-05
2330 RSI crossover
2016-09-06
延長標的持有時間
2330 ma crossover
2016-09-07
延長標的持有時間
2016-09-08
2016-09-09
2016-09-10
2016-09-12
2016-09-13
2016-09-14
2016-09-19
2330 RSI crossover
2016-09-20
延長標的持有時間
2016-09-21
2016-09-22
2330 ma crossover
2016-09-23
延長標的持有時間
2016-09-26
2016-09-29
2330 RSI crossover
2016-09-30
延長標的持有時間
2016-10-03
2330 RSI cro

2019-01-03
2019-01-04
2019-01-07
2019-01-08
2019-01-09
2330 RSI crossover
2019-01-10
延長標的持有時間
2019-01-11
2019-01-14
2019-01-15
2330 ma crossover
2019-01-16
延長標的持有時間
2019-01-17
2019-01-18
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-28
2019-01-29
2019-01-30
2330 2018-10-17 00:00:00 2019-01-30 00:00:00 , entry/exit price: 241.5 220.5
equity change: -212604.0
2019-02-11
2330 RSI crossover
2019-02-12
date: 2019-02-12, 買進股票2330, price: entry_price
2019-02-13
2019-02-14
2019-02-15
2019-02-18
2330 RSI crossover
2019-02-19
延長標的持有時間
2019-02-20
2019-02-21
2019-02-22
2019-02-25
2019-02-26
2019-02-27
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2330 2019-02-12 00:00:00 2019-03-07 00:00:00 , entry/exit price: 230.0 235.0
equity change: 48530.0
2019-03-08
2019-03-11
2019-03-12
2330 RSI crossover
2019-03-13
date: 2019-03-13, 買進股票2330, price: entry_price
2019-03-14
2019-03-15
2330 RSI crossover
2330 ma crossover
已經有待處理的委託（2330）。
2019-03-18
延長標的持有時間
2019-03-19
2019-03-20
2019-03-21
201

In [10]:
account.equity

3862928.8000000007

In [11]:
1 / 1.012 ** 30

0.6991729615376817